### Installation

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab and Kaggle notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
    !pip install --no-deps cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###  Using Unsloth in fine tuning  LLm

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 
dtype = None 
#4bit quantization
load_in_4bit = True .

# model = [
#     "unsloth/Meta-Llama-3.1-8B-bnb-4bit",     

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/content/drive/MyDrive/second_trial_weights-20250303T091315Z-001/second_trial_weights", #  previous trained weights
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
  
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

Unsloth 2025.2.15 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [4]:
#LoRA configuration
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, 
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, 
    bias = "none",    
    use_gradient_checkpointing = "unsloth", 
    random_state = 3407,
    use_rslora = False,  
    loftq_config = None, 
)

Unsloth: Already have LoRA adapters! We shall skip this step.


In [6]:
# data loading and formating
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("MaryWambo/combined_afri_JW300", split = "train")

README.md:   0%|          | 0.00/83.0 [00:00<?, ?B/s]

cleaned_combined.json:   0%|          | 0.00/13.0M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
#data conversion using  standardize_sharegpt
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)

Standardizing format:   0%|          | 0/29971 [00:00<?, ? examples/s]

Map:   0%|          | 0/29971 [00:00<?, ? examples/s]

We look at how the conversations are structured for item 5:

In [8]:
dataset[5]["conversations"]

[{'content': 'depending on the variety and the climate conditions.',
  'role': 'user'},
 {'content': 'Kũringana na mĩthemba ya mĩmera ĩyo na ũrĩa rĩera rĩhaana.',
  'role': 'assistant'}]

In [9]:
dataset[5]["text"]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\ndepending on the variety and the climate conditions.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nKũringana na mĩthemba ya mĩmera ĩyo na ũrĩa rĩera rĩhaana.<|eot_id|>'

<a name="Train"></a>
### Train the model


In [10]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1,
        # max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Converting train dataset to ChatML (num_proc=2):   0%|          | 0/29971 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/29971 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/29971 [00:00<?, ? examples/s]

Truncating train dataset (num_proc=2):   0%|          | 0/29971 [00:00<?, ? examples/s]

In [11]:
#using train_on_completions method to only train on the assistant outputs
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map:   0%|          | 0/29971 [00:00<?, ? examples/s]

We verify masking is actually done:

In [12]:
#decoding
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\ndepending on the variety and the climate conditions.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nKũringana na mĩthemba ya mĩmera ĩyo na ũrĩa rĩera rĩhaana.<|eot_id|>'

In [13]:
#masking out the user inputs
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                            \n\nKũringana na mĩthemba ya mĩmera ĩyo na ũrĩa rĩera rĩhaana.<|eot_id|>'

In [14]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 29,971 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 3,746
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss
1,0.766000
2,0.790400
3,0.630800
4,0.798800
5,0.617300
6,0.458500
7,0.466000
8,0.839200
9,0.760100
10,0.628400


In [15]:
save_path = "/content/drive/MyDrive/third_trial_weights"

In [16]:
#saving the trained model
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

('/content/drive/MyDrive/third_trial_weights/tokenizer_config.json',
 '/content/drive/MyDrive/third_trial_weights/special_tokens_map.json',
 '/content/drive/MyDrive/third_trial_weights/tokenizer.json')

In [1]:
!pip install sacrebleu

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 KB 315.9 kB/s eta 0:00:001m300.4 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 KB 803.2 kB/s eta 0:00:00 kB/s eta 0:00:01:01


In [31]:
#evaluating the performance of the model
import sacrebleu
generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
reference_texts = [["we should repent ours sins daily"]]
bleu = sacrebleu.corpus_bleu([generated_text], reference_texts)
print(f"BLEU Score: {bleu.score}")

BLEU Score: 13.52328506650155


<a name="Inference"></a>
### Inferencing the model


In [30]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) 

messages = [
    {"role": "user", "content": "we should repent ours sins daily"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, 
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
tokenizer.batch_decode(outputs)

['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nwe should repent ours sins daily<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\ntwagĩrĩirwo gũcokeria mehia maitũ o mũthenya<|eot_id|>']